<h1>Grupo GCPI - Modelo preditivo Rappi</h1>

In [ ]:
#@title Utilização da aplicação {run: "auto"}
while True:
    import pandas as pd
    import plotly.express as px
    import numpy as np
    import datetime
    from operator import index
    import warnings
    from termcolor import colored
    warnings.filterwarnings('ignore')
    from google.colab import drive
    drive.mount('/content/drive')
    from joblib import load
    modelo_lgbm = load('/content/drive/MyDrive/Rappi/modelos/modelo_lgbm.model')
    modelo_lr = load('/content/drive/MyDrive/Rappi/modelos/modelo_logistic_regression.model')
    scaler = load('/content/drive/MyDrive/Rappi/modelos/scaler_modelo_lgbm.scaler')
    objetivo = "Maior certeza do resultado" #@param ["Maior numero de possiveis churn", "Maior certeza do resultado"]
    if objetivo == 'Maior numero de possiveis churn':
        modelo = modelo_lr
        modelo_print = "_modelo_lr"
    else:
        modelo = modelo_lgbm
        modelo_print = "_modelo_lgbm"

    try:
        print("Carregando tabela infos_gerais...")
        df = pd.read_csv("/content/drive/MyDrive/Rappi/bases/infos gerais.csv", usecols={'ID', 'ULTIMO_PEDIDO'})
        df[['Date', 'exclude']] = df['ULTIMO_PEDIDO'].str.split('T', expand=True)
        df['Date']= df['Date'].astype('datetime64[ns]')
        df['Date'].max()
        df['Diferenca_Dias'] = df['Date'].max() - df['Date']
        for i in  df.select_dtypes(include=['datetime64']).columns.tolist():
            df['Diferenca_Dias'] = df['Diferenca_Dias'].astype(str)
        df[['dif_num', 'exclude_3']] = df['Diferenca_Dias'].str.split(' ', expand=True)
        df['CHURN'] = df['dif_num'].astype(int)
        df['CHURN'] = np.where(df['CHURN'] < 21, 0, df['CHURN'])
        df['CHURN'] = np.where(df['CHURN'] >= 21, 1, df['CHURN'])
        df = df.drop(['ULTIMO_PEDIDO', 'Date', 'exclude', 'Diferenca_Dias', 'dif_num', 'exclude_3'], axis=1)
        df_infos = pd.read_csv("/content/drive/MyDrive/Bases/infos gerais.csv", usecols=['ID', 'AUTO_ACEITE', 'GORJETA', 'FRETE_MEDIO', 'TRANSPORTE', 'DATA_NASCIMENTO', 'CIDADE'])
        df_infos
        df_infos[['YEAR', 'MONTH', 'DAY']] = df_infos['DATA_NASCIMENTO'].str.split('-', expand=True)
        date_now = datetime.datetime.now()
        year_now = date_now.strftime('%Y')
        df_infos['YEAR'] = df_infos['YEAR'].astype(int)
        df_infos['AGE'] = int(year_now) - df_infos['YEAR']
        df_infos.drop(['DATA_NASCIMENTO', 'YEAR', 'MONTH', 'DAY'], inplace=True, axis=1)
        df_infos = df_infos.query("TRANSPORTE == 'car' or  TRANSPORTE == 'bicycle' or TRANSPORTE == 'motorbike'")
        df_infos = pd.get_dummies(df_infos, columns=["TRANSPORTE"])
        df_infos.loc[df_infos['AUTO_ACEITE'] == True, 'AUTO_ACEITE'] = 1
        df_infos.loc[df_infos['AUTO_ACEITE'] == False, 'AUTO_ACEITE'] = 0
        df_infos['CIDADE'] = df_infos['CIDADE'].replace(['Sao Paulo', 'Natal', 'Campinas', 'Recife',
            'Fortaleza', 'Brasilia', 'Sorocaba', 'Salvador', 'Cotia', 'Santos',
            'São Jose', 'Goiânia',
            'Ribeirão Preto', 'Porto Alegre', 'Curitiba', 'Pinhais', 'Niterói',
            'Diadema', 'Teresina', 'Londrina', 'Guarujá', 'Balneario Camboriú',
            'Jundiaí', 'Florianópolis', 'Betim', 'São José do Rio Preto',
            'Praia Grande', 'João Pessoa', 'Hortolândia', 'Nova Iguacu',
            'Juiz de Fora', 'Maceió', 'São João de Meriti', 'Canoas', 'Poá',
            'Uberlandia', 'São Roque', 'Vila Velha', 'Maua', 'Aracaju',
            'Paulínia', 'Suzano', 'Novo Hamburgo', 'Parnamirim', 'Belém',
            'Lauro de Freitas', 'Valparaíso de Goiás', 'Olinda', '14', '10',
            'Jaboatao Dos Guararapes', 'São José dos Campos', 'Itaguaí',
            'Itajaí', 'Nilópolis', 'Vinhedo', 'Duque De Caxias', 'Petrópolis',
            'Palhoça', 'São Vicente', 'Barueri', 'São José Dos Pinhais',
            'Valinhos', 'Jacareí', 'Ipatinga', 'Itupeva', '18', '199',
            'Taubaté', 'Sumaré', 'São Gonçalo', 'Rio das Ostras', 'Vassouras',
            'Mogi Das Cruzes', '24', '29', 'Ponta Grossa', 'Votorantim', 'Itu',
            'Matinhos', 'Teófilo Otoni', 'Mesquita', 'Maricá', 'Piracicaba',
            'Vitória', '11', 'Santo Amaro da Imperatriz', 'Atibaia',
            'Indaiatuba', '26', 'Barreiras', 'Itabuna', 'Passo Fundo', 'Bauru',
            'Sertãozinho', 'Petrolina', 'Ananindeua', 'Canasvieiras', '31',
            'Cabo Frio', 'Montes Claros', 'Caruaru', 'Rondonópolis',
            'Full Coverage', 'Itapema', 'Anápolis', 'Gramado',
            'Poços de Caldas', 'Campo Mourão', 'Feira de Santana', '79',
            'São Luis', 'Ourinhos', '61', 'Araraquara', 'Araçatuba',
            'Americana', 'Manaus', 'Campina Grande', 'Cariacica', '32',
            'Porto Velho', 'Presidente Prudente', 'Pouso Alegre', '30',
            'Blumenau', '51', 'Resende', 'Mogi Guaçu', 'Bertioga',
            'Balneario Camboriu', 'Vitória da Conquista', 'Nova Friburgo',
            'Jaraguá do Sul', '146', 'Sao Jose Dos Pinhais', '5',
            'Teresópolis', 'Torres', 'Macaé', '95', 'Sao Vicente', '50',
            'Mossoró', 'Pirassununga', 'Macapá', 'Campos dos Goytacazes',
            'São Paulo', 'Campo Grande', '282', 'Franca', 'São Carlos',
            'Cascavel', 'Imperatriz', 'Rio Branco', '81', 'Xangrila',
            'Uberaba', 'Joinville', 'Porto Seguro', 'Sumare', '76', 'Sao Jose',
            '48', 'Maringá', '62', 'Caldas Novas', 'Sao Luis', '33', 'Maringa',
            'Volta Redonda', 'Limeira', 'Santa Maria', 'Governador Valadares',
            'Foz do Iguaçu', 'without_city', '60', 'Rio Grande', '161', '47',
            '218', '74', '9', 'FAZENDA RIO GRANDE', 'Playa del Carmen',
            'Outras cidades', '94', '156', 'Marilia', 'Sao Goncalo', '49',
            '233', 'Sao Carlos', 'SÃO PAULO', 'BELO HORIZINTE',
            'RIO DE JANEIRO', '154', '75', '59', '147', 'Cuiabá', '130', '97',
            '78', '321', 'Caxias do Sul', 'Full Coverage Brasil', '159',
            'Osasco', '16', '252', '149', '13', '12', 'Boa Vista', '126',
            '289', 'Rosarito', 'Mexicali', 'Tuxtla Gutierrez', 'Viña del Mar',
            'Juliaca', 'Marília', 'Serra', 'Rio Claro'],'Outros')
        df_infos = pd.get_dummies(df_infos, columns=['CIDADE'])
        df = df.merge(df_infos, how='inner', left_on= 'ID', right_on='ID')
        print(colored("!!!!Sucesso ao obter tabela infos_gerais!!!!", "green"))
        print("Carregando tabela Orders Done e Cancel...")
    except:
        print(colored("!!!!Erro ao obter tabela infos_gerais!!!!", "red"))
        break

    try:
        df_orders = pd.read_csv("/content/drive/MyDrive/Rappi/bases/Ordens Done e Cancel.csv", dtype={"STOREKEEPER_ID": "int32", "ORDERS_DONE": "int16", "ORDERS_CANCEL": "int16",  "CANCELS_OPS_RT": "int16"})
        df_orders['CANCEL_RATE'] = (df_orders['ORDERS_CANCEL'] + df_orders['CANCELS_OPS_RT']) / (df_orders['ORDERS_DONE'] + df_orders['ORDERS_CANCEL'] + df_orders['CANCELS_OPS_RT'])
        df_orders = df_orders.drop(columns=['ORDERS_CANCEL', 'CANCELS_OPS_RT', 'ORDERS_DONE'])
        df = df.merge(df_orders, how='inner', left_on= 'ID', right_on='STOREKEEPER_ID')
        df.drop('STOREKEEPER_ID', inplace=True, axis=1)
        df_orders = ''
        print(colored("!!!!Sucesso ao obter tabela Ordens Done e Cancel!!!!", "green"))
        print("Carregando tabela Incidentes_Regras RT...")
    except:
        print(colored("Erro ao obter tabela Ordens Done e Cancel", "red"))
        break

    try:
        df_incidentes = pd.read_csv("/content/drive/MyDrive/Rappi/bases/Incidentes_Regras RT.csv", usecols=['STOREKEEPER_ID','PUNISHMENT_MINUTES'], dtype={"STOREKEEPER_ID": "int32", "PUNISHMENT_MINUTES": "int32"})
        df_incidentes = df_incidentes.groupby('STOREKEEPER_ID')['PUNISHMENT_MINUTES'].agg(['sum','count'])
        df_incidentes['STOREKEEPER_ID'] = df_incidentes.index
        df_incidentes.rename(columns = {'sum':'MINUTES_PUNISHMENTS', 'count':'NUM_PUNISHMENTS'}, inplace = True)
        df_incidentes = df_incidentes.reset_index(drop=True)
        df_incidentes = df_incidentes.astype(int)
        df = df.merge(df_incidentes, how='left', left_on= 'ID', right_on='STOREKEEPER_ID')
        df.drop('STOREKEEPER_ID', inplace=True, axis=1)
        df[['MINUTES_PUNISHMENTS', 'NUM_PUNISHMENTS']] = df[['MINUTES_PUNISHMENTS', 'NUM_PUNISHMENTS']].fillna(0)
        df_incidentes = ''
        indexNames = df[df['MINUTES_PUNISHMENTS'] >= 21600000].index
        df = df.drop(indexNames)
        indexNames = ""
        print(colored("!!!!Sucesso ao obter tabela Incidentes_Regras RT!!!!", "green"))
        print("Carregando tabela Distance...")
    except:
        print(colored("Erro ao obter tabela Incidentes_Regras RT", "red"))
        break

    try:
        df_distance = pd.read_csv("/content/drive/MyDrive/Rappi/bases/Distance.csv", usecols=['STOREKEEPER_ID', 'DISTANCE_TO_USER'])
        df_distance = df_distance.dropna()
        df_distance['STOREKEEPER_ID'].isna().sum()
        df_distance['STOREKEEPER_ID'] = df_distance['STOREKEEPER_ID'].astype(int)
        deliveries_count = df_distance['STOREKEEPER_ID'].value_counts()
        deliveries_count = deliveries_count.to_frame()
        deliveries_count.rename(columns={'STOREKEEPER_ID': 'DELIVERIES_COUNT'}, inplace = True)
        df_distance = df_distance.groupby('STOREKEEPER_ID').sum()
        distance_total = deliveries_count.merge(df_distance, how='inner', left_index=True, right_on='STOREKEEPER_ID')
        distance_total['DISTANCE_MEDIAN'] = distance_total['DISTANCE_TO_USER']/distance_total['DELIVERIES_COUNT']
        distance_total = distance_total.round({"DISTANCE_MEDIAN":1})
        distance_total.drop('DISTANCE_TO_USER', inplace=True, axis=1)
        distance_total['STOREKEEPER_ID'] = distance_total.index
        distance_total = distance_total.reset_index(drop=True)
        df = df.merge(distance_total, how='inner', left_on= 'ID', right_on='STOREKEEPER_ID')
        df.drop('STOREKEEPER_ID', inplace=True, axis=1)
        df_distance = ''

        df_frete = df[['ID','FRETE_MEDIO']]
        df_frete.rename(columns={'ID': 'STOREKEEPER_ID'}, inplace = True)
        df_frete.rename(columns={'FRETE_MEDIO': 'SHIPPING_MEDIAN'}, inplace = True)
        Rts_frete_per_distance = df_frete.merge(distance_total, how='inner', left_on='STOREKEEPER_ID', right_on='STOREKEEPER_ID')
        Rts_frete_per_distance = Rts_frete_per_distance.drop_duplicates(keep='first')
        Rts_frete_per_distance['SHIPPING_PER_DISTANCE'] = Rts_frete_per_distance['SHIPPING_MEDIAN']/Rts_frete_per_distance['DISTANCE_MEDIAN']
        Rts_frete_per_distance.drop(['DELIVERIES_COUNT', 'DISTANCE_MEDIAN', 'SHIPPING_MEDIAN'], inplace=True, axis=1)
        df = df.merge(Rts_frete_per_distance, how='inner', left_on= 'ID', right_on='STOREKEEPER_ID')
        df.drop('STOREKEEPER_ID', inplace=True, axis=1)
        df.drop('FRETE_MEDIO', inplace=True, axis=1)
        df.drop('DISTANCE_MEDIAN', inplace=True, axis=1)
        Rts_frete_per_distance = ''
        df_frete = ''
        df_infos = ''
        print(colored("!!!!Sucesso ao obter tabela Distance!!!!", "green"))
        print("Processando dados e carregando modelos...")
    except:
        print(colored("Erro ao obter tabela Distance", "red"))
        break

    try:
        df = df.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
        df = df.dropna()
        df['AUTO_ACEITE'] = df['AUTO_ACEITE'].astype('int8')
        colunas = ['GORJETA', 'AGE', 'MINUTES_PUNISHMENTS', 'NUM_PUNISHMENTS', 'DELIVERIES_COUNT', 'SHIPPING_PER_DISTANCE']
        x = df.drop(columns=['ID', 'CHURN'])
        x[colunas] = scaler.transform(x[colunas])
        lista = modelo.predict_proba(x)
        x['churn'] = modelo.predict(x)
        df['churn'] = x['churn']
        df = df.drop(columns=['AUTO_ACEITE',	'GORJETA',	'AGE',	'TRANSPORTE_bicycle',	'TRANSPORTE_car',	'TRANSPORTE_motorbike',	'CIDADE_Belo Horizonte',	'CIDADE_Grande São Paulo',	'CIDADE_Outros'	,'CIDADE_Rio de Janeiro',	'CANCEL_RATE'	,'MINUTES_PUNISHMENTS'	,'NUM_PUNISHMENTS'	,'DELIVERIES_COUNT',	'SHIPPING_PER_DISTANCE'])
        df.reset_index(drop=True, inplace=True)
        array_vazio= []
        for array_pequenas in lista:
            if array_pequenas[1] <= 0.2:
                array_vazio.append(1)
            elif array_pequenas[1] > 0.2 and array_pequenas[1] <= 0.4:
                array_vazio.append(2)
            elif array_pequenas[1] > 0.6 and array_pequenas[1] <= 0.8:
                array_vazio.append(4)
            elif array_pequenas[1] > 0.8:
                array_vazio.append(5)
            else: 
                array_vazio.append(3)
        new_df = pd.DataFrame(array_vazio)
        df["PROBABILIDADE_CHURN"] = new_df[0]
        df['FILTRO'] = df['churn'] - df['CHURN']
        df = df.sort_values(by=['FILTRO'], ascending=False)
        df = df.drop(columns=['FILTRO'])
        df = df.replace({'PROBABILIDADE_CHURN': {1: 'Muito Baixa', 2: 'Baixa', 3: 'Média', 4: 'Alta', 5: 'Muito Alta'}})
        df.columns = ['ID', 'RT_INATIVO', 'PREVISTO_CHURN', 'PROBABILDADE_CHURN']
        df = df.replace({'RT_INATIVO': {0: 'Não', 1: 'Sim'}, 'PREVISTO_CHURN': {0: 'Não', 1: 'Sim'}})
        print(colored("!!!!Sucesso ao processar dados e aplicar o modelo!!!! \n", "green"))
    except:
        print(colored("Erro ao processar dados e aplicar o modelo \n", "red"))
        break

    import ipywidgets as widgets
    from IPython.display import display
    button_csv = widgets.Button(description="Exportar CSV")
    button_excel = widgets.Button(description="Exportar XLSX")
    output_csv = widgets.Output()
    output_excel = widgets.Output()
    def on_button_clicked_csv(b):
        with output_csv:
            print("Aguarde a conversão...")
            df.to_csv('/content/drive/MyDrive/Rappi/churn' + modelo_print + '.csv', index=False)
            print(colored("!!!!CSV criado com sucesso na pasta do projeto!!!! \n", "green"))
    def on_button_clicked_excel(b):
        with output_excel:
            print("Aguarde a conversão...")
            df.to_excel('/content/drive/MyDrive/Rappi/churn' + modelo_print + '.xlsx', index=False)
            print(colored("!!!!Planilha criada com sucesso na pasta do projeto!!!! \n", "green"))
    button_csv.on_click(on_button_clicked_csv)
    button_excel.on_click(on_button_clicked_excel)
    display(button_csv, output_csv)
    display(button_excel, output_excel)
    break